In [1]:
from google.colab import files
uploaded = files.upload()

Saving dense_dataset_medianfillna.csv to dense_dataset_medianfillna.csv
Saving test_na_new.csv to test_na_new.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import cv

In [3]:
dense_median_nona = pd.read_csv('dense_dataset_medianfillna.csv')
print(dense_median_nona.shape)
dense_median_nona #feature output

(7715, 132)


,Id,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,...,P15.2,P16.2,P17.2,P18.2,P19.2,P20.2,P21.2,P22.2,P23.2,P24.2,P25.2,P26.2,P27.2,P1.3,P2.3,P3.3,P4.3,P5.3,P6.3,P7.3,P8.3,P9.3,P10.3,P11.3,P12.3,P13.3,P14.3,P15.3,P16.3,P17.3,P18.3,P19.3,P20.3,P21.3,P22.3,P23.3,P24.3,P25.3,P26.3,P27.3
0,-9173346003017351704,0.538462,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,24502.0,0.040000,3.729679,33.883333,10.500000,136.020000,0.140000,0.080000,0.140000,0.260000,4.89,0.100000,1983.480769,81.250000,0.200000,9.224319,...,98.050000,0.111111,6.883721,14477.962963,45.850000,4.93,13.495062,301.0,0.000000,4.930000,0.740741,3053.0,0.296296,58.0,24502.0,0.114286,3.729679,33.883333,10.500000,136.020000,0.14,0.080000,0.185185,0.285714,4.93,0.057143,2195.027027,81.250000,0.200000,9.040698,11610.160000,27.600000,4.93,9.931333,245.0,0.000000,4.930000,0.740000,2117.0,0.296296
1,-9152034383903842080,0.025641,1.0,0.000000,0.149277,0.006757,0.000000,0.270289,0.123902,0.013514,0.027778,0.015625,0.198527,0.000000,0.000000,0.000000,0.000000,0.006757,0.000000,0.000000,0.174062,0.013514,0.000000,85.0,14823.0,0.733333,2.674660,30.850000,16.650000,118.733333,0.000000,0.000000,0.266667,0.266667,4.92,0.000000,1483.352941,56.500000,0.400000,4.492255,...,87.566667,0.125000,6.143172,11236.750000,23.966667,4.92,9.494792,228.0,0.000000,4.920000,0.250000,1728.0,0.437500,68.0,43364.0,0.837838,2.839502,63.100000,15.333333,159.081081,0.00,0.027027,0.081081,0.297297,4.91,0.054054,1527.853659,499.733333,0.243243,7.462687,11961.540541,37.050000,4.92,10.445495,500.0,0.027027,4.916486,0.108108,3507.0,0.324324
2,-9147768078838301331,0.487179,14.0,0.004167,0.013384,0.056865,0.050726,0.009486,0.017298,0.144919,0.000000,0.348029,0.000000,0.008333,0.074244,0.102604,0.135981,0.000000,0.011979,0.000000,0.021986,0.000000,0.000000,70.0,35260.0,0.451613,2.991992,39.733333,9.850000,162.838710,0.000000,0.000000,0.129032,0.096774,4.95,0.096774,1519.054054,181.450000,0.322581,9.688581,...,172.183333,0.255319,6.119048,10138.127660,24.300000,4.95,8.535816,481.0,0.000000,4.950000,0.595745,1511.0,0.425532,70.0,45466.0,0.366667,2.855353,53.566667,7.100000,163.400000,0.00,0.000000,0.200000,0.133333,4.90,0.100000,1326.275000,487.000000,0.233333,7.375000,11111.900000,54.800000,4.95,9.855556,450.0,0.016667,4.938500,0.616667,1469.0,0.333333
3,-9136193043722118347,0.000000,0.0,0.026147,0.012989,0.023077,0.000000,0.000000,0.088023,0.025641,0.000000,0.180027,0.000000,0.000000,0.032692,0.104521,0.051619,0.092982,0.237078,0.000000,0.116869,0.000000,0.008333,86.0,35866.0,0.026316,3.548534,37.833333,18.116667,160.000000,0.000000,0.000000,0.051282,0.282051,4.97,0.000000,1585.255814,84.600000,0.256410,10.000000,...,73.450000,0.384615,15.519348,11353.307692,20.616667,4.96,7.832692,681.0,0.038462,4.960000,0.923077,3556.0,0.230769,88.0,48100.0,0.100000,3.620698,40.383333,11.016667,191.233333,0.00,0.000000,0.100000,0.066667,4.95,0.000000,1682.468750,100.316667,0.266667,9.503106,15764.233333,24.000000,4.96,7.956111,417.0,0.066667,4.955333,0.833333,3005.0,0.566667
4,-9124955046829186109,0.054054,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,18735.0,0.121212,3.268748,22.900000,11.300000,104.181818,0.060606,0.060606,0.121212,0.090909,4.97,0.393939,1331.282051,39.200000,0.272727,7.454545,...,39.200000,0.200000,4.349451,9661.800000,11.433333,4.97,7.513333,233.0,0.000000,4.970000,0.200000,2857.0,0.400000,140.0,22157.0,0.800000,1.954941,43.183333,21.316667,205.400000,0.00,0.000000,0.000000,0.400000,4.97,0.000000,777.600000,79.516667,0.000000,4.349451,12713.800000,10.466667,4.97,7.513333,332.0,0.000000,4.970000,0.200000,5506.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [4]:
test_dense_median_nona = pd.read_csv('test_na_new.csv')
print(test_dense_median_nona.shape)
test_dense_median_nona #feature output

(3307, 131)


,Id,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,...,P15.2,P16.2,P17.2,P18.2,P19.2,P20.2,P21.2,P22.2,P23.2,P24.2,P25.2,P26.2,P27.2,P1.3,P2.3,P3.3,P4.3,P5.3,P6.3,P7.3,P8.3,P9.3,P10.3,P11.3,P12.3,P13.3,P14.3,P15.3,P16.3,P17.3,P18.3,P19.3,P20.3,P21.3,P22.3,P23.3,P24.3,P25.3,P26.3,P27.3
0,-9193687728771344321,0.102564,3.0,0.004261,0.040318,0.016337,0.000000,0.540997,0.151378,0.006642,0.011905,0.025667,0.013267,0.000000,0.010938,0.108358,0.004193,0.000000,0.000000,0.000000,0.065740,0.000000,0.000000,79.0,46682.0,0.314286,3.987764,28.566667,7.500000,169.561905,0.000000,0.000000,0.152381,0.200000,5.00,0.009524,1945.815789,84.616667,0.200000,11.982507,...,47.783333,0.214286,8.492683,7873.952381,22.300000,5.00,8.453571,261.0,0.000000,5.000000,0.333333,1173.0,0.404762,53.0,38504.0,0.410448,3.697902,25.850000,7.233333,159.293233,0.000000,0.000000,0.195489,0.255639,5.00,0.007519,1775.828025,59.483333,0.187970,12.252964,8397.721805,48.483333,5.00,9.042910,507.0,0.014925,5.000000,0.574627,1414.0,0.353383
1,-9190465597412250947,0.051282,1.0,0.000000,0.016051,0.000000,0.004310,0.076665,0.029991,0.003788,0.000000,0.245169,0.000000,0.000000,0.183220,0.316674,0.000000,0.000000,0.000000,0.000000,0.100561,0.023569,0.000000,47.0,24735.0,0.189655,2.717834,22.750000,8.400000,101.551724,0.000000,0.000000,0.086207,0.086207,4.96,0.396552,1070.352941,62.800000,0.379310,7.256637,...,51.100000,0.196970,8.072917,5928.803030,16.650000,4.96,6.524495,205.0,0.000000,4.960000,0.848485,1070.0,0.030303,47.0,12598.0,0.228571,2.437603,24.583333,8.833333,98.194444,0.000000,0.000000,0.111111,0.166667,4.96,0.416667,1095.463415,51.300000,0.277778,6.658291,5984.111111,16.700000,4.96,7.720476,162.0,0.000000,4.960000,0.771429,2312.0,0.027778
2,-9181694933618130417,0.025641,1.0,0.014156,0.011477,0.003333,0.000000,0.197212,0.162136,0.003906,0.000000,0.003333,0.000000,0.000000,0.000000,0.011146,0.000000,0.202684,0.124655,0.000000,0.262627,0.003333,0.000000,45.0,31446.0,0.578125,2.759339,33.266667,10.016667,122.296875,0.234375,0.109375,0.171875,0.156250,4.97,0.281250,1312.662338,74.300000,0.187500,7.084469,...,95.400000,0.266667,8.974359,8358.546667,37.783333,4.97,7.634444,337.0,0.013333,4.965333,0.106667,1533.0,0.160000,55.0,20112.0,0.551724,3.178472,35.533333,12.083333,152.474576,0.068966,0.034483,0.101695,0.271186,4.96,0.050847,1314.186667,80.066667,0.186441,8.549618,9389.525424,17.266667,4.96,7.970402,273.0,0.000000,4.960000,0.344828,1684.0,0.389831
3,-9173096570645067699,0.051282,1.0,0.026021,0.212603,0.036445,0.007576,0.007576,0.070076,0.096063,0.033151,0.007576,0.022282,0.062466,0.000000,0.022282,0.018445,0.022282,0.029412,0.000000,0.186073,0.128802,0.010870,47.0,33198.0,0.750000,3.112121,115.050000,19.350000,168.500000,0.000000,0.000000,0.000000,0.000000,5.00,0.250000,1570.555556,487.016667,0.000000,10.253623,...,239.316667,0.000000,8.252874,18716.294118,12.766667,5.00,7.847059,402.0,0.058824,5.000000,0.352941,3134.0,0.705882,60.0,52474.0,0.434783,3.786407,91.883333,10.666667,257.608696,0.000000,0.000000,0.000000,0.043478,5.00,0.086957,1818.785714,487.016667,0.086957,7.267206,22840.695652,43.633333,5.00,9.690580,425.0,0.086957,5.000000,0.478261,1618.0,0.782609
4,-9167448628699201489,0.041667,1.0,0.002577,0.032492,0.000000,0.002941,0.252257,0.156198,0.019296,0.000000,0.040430,0.000000,0.000000,0.005518,0.122393,0.002577,0.030322,0.050056,0.002577,0.246354,0.034011,0.000000,64.0,21273.0,0.148148,2.560541,29.050000,9.066667,125.384615,0.000000,0.000000,0.115385,0.192308,4.99,0.153846,1056.806452,47.366667,0.269231,8.734234,...,73.516667,0.329412,8.599034,9741.941176,21.983333,4.99,8.548810,318.0,0.000000,4.990000,0.857143,2202.0,0.329412,80.0,41680.0,0.231579,2.909809,36.533333,8.016667,150.610526,0.010526,0.010526,0.178947,0.200000,4.99,0.021053,1340.535714,490.000000,0.273684,7.485714,8773.178947,44.450000,4.99,8.497544,374.0,0.000000,4.990000,0.747368,1844.0,0.326316
...,...,..

In [5]:
X_train = dense_median_nona.drop(['target', 'Id'], axis = 1)
Y_train = dense_median_nona['target']

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)

In [6]:
X_test = test_dense_median_nona.drop(['Id'], axis = 1)

In [ ]:
xgtrain = xgb.DMatrix(X_train, label=Y_train)
model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', learning_rate=0.300000012, max_delta_step=0, max_depth=12,
              min_child_weight=1, num_parallel_tree=5, n_estimators=100, seed=9, gamma=0, eta=0.1, colsample_bytree=0.5, sampling_method='gradient_based')
xgb_param = model.get_xgb_params()
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=1200, nfold=10, metrics=['auc'],
     early_stopping_rounds=50, stratified=True, seed=9)
print('Best number of trees = {}'.format(cvresult.shape[0]))
model.set_params(n_estimators=cvresult.shape[0])
print('Fit on the trainings data')
model.fit(X_train, Y_train, eval_metric='auc')

In [15]:
print('Overall AUC:', roc_auc_score(Y_train, model.predict_proba(X_train)[:,1]))

Overall AUC: 1.0


In [16]:
Y_pred = model.predict(X_train, ntree_limit=cvresult.shape[0])
Y_pred

array([1., 0., 0., ..., 0., 0., 0.])

In [17]:
accuracy = accuracy_score(Y_train, Y_pred)
print('Accuracy on training data: ', accuracy)

Y_pred_val = model.predict(X_val)
accuracy = accuracy_score(Y_val, Y_pred_val)
print('Accuracy on validation data: ', accuracy)

pred_test = model.predict_proba(X_test, ntree_limit=cvresult.shape[0])

Accuracy on training data:  1.0
Accuracy on validation data:  0.9300215982721383


In [12]:
pred_test = pd.DataFrame(data = pred_test)

In [13]:
pred_test[0].shape[0]

3307

In [14]:
id = test_dense_median_nona['Id']
result = pd.concat([id, pred_test], axis=1)
result = result.drop(result.columns[1], axis=1)
result = result.rename(columns={1 : 'Predicted'})
result

,Id,Predicted
0,-9193687728771344321,0.000048
1,-9190465597412250947,0.000016
2,-9181694933618130417,0.000005
3,-9173096570645067699,0.001607
4,-9167448628699201489,0.000003
...,...,...
3302,9196194430635355476,0.000007
3303,9205406046009482646,0.000055
3304,9206342053685832016,0.002638
3305,9213027496842209017,0.000106


In [18]:
result.to_csv('result.csv', index=False) 
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>